In [1]:
# aivietnam.ai
# sum-area table ; simple implementation
import numpy as np
from PIL import Image
import math

def stereo_matching_corr(left_img, right_img, kernel_size, disparity_range):
   
    # đọc ảnh trái và ảnh phải, rồi chuyển sang ảnh grayscale
    left_img  = Image.open(left_img).convert('L')
    left      = np.asarray(left_img)
    
    right_img = Image.open(right_img).convert('L')
    right     = np.asarray(right_img) 
       
    # cho trước chiều rộng và chiều cao của ảnh
    height = 370 #288
    width  = 427 #384
    
    # tạo disparity map
    depth = np.zeros((height, width), np.uint8)   
    
    n = kernel_size*kernel_size
    kernel_half = int( (kernel_size-1) / 2)    
    scale = 255 / disparity_range
    
    # build sum-area table for each disparity
    corr_space = np.zeros((disparity_range, height, width))
    for j in range(disparity_range):
        print(".", end=" ")
        
        memory_sum_x  = np.ones((height, width))
        memory_sum_y  = np.ones((height, width))
        memory_sum_xy = np.ones((height, width))
        memory_sum_x2 = np.ones((height, width))
        memory_sum_y2 = np.ones((height, width))
    
        data_sum_x  = np.ones((height, width))*255
        data_sum_y  = np.ones((height, width))
        data_sum_xy = np.ones((height, width))*255
        data_sum_x2 = np.ones((height, width))*255*255
        data_sum_y2 = np.ones((height, width))
        
        for y in range(kernel_half, height-kernel_half):
            for x in range(kernel_half, width-kernel_half):
                if (x - j >= 0):
                    data_sum_x[y, x]  = int(left[y, x])
                    data_sum_y[y, x]  = int(right[y, x - j])
                    data_sum_xy[y, x] = int(left[y, x])*int(right[y, x - j])
                    data_sum_x2[y, x] = int(left[y, x])*int(left[y, x])
                    data_sum_y2[y, x] = int(right[y, x - j])*int(right[y, x - j])
        
        # first item
        memory_sum_x[0, 0]  = data_sum_x[0, 0]
        memory_sum_y[0, 0]  = data_sum_y[0, 0]
        memory_sum_xy[0, 0] = data_sum_xy[0, 0]
        memory_sum_x2[0, 0] = data_sum_x2[0, 0]
        memory_sum_y2[0, 0] = data_sum_y2[0, 0]
    
        # first row
        for y in range(1):
            for x in range(1, width):
                memory_sum_x[y, x]  = memory_sum_x[y, x - 1] + data_sum_x[y, x]
                memory_sum_y[y, x]  = memory_sum_y[y, x - 1] + data_sum_y[y, x]
                memory_sum_xy[y, x] = memory_sum_xy[y, x - 1] + data_sum_xy[y, x]
                memory_sum_x2[y, x] = memory_sum_x2[y, x - 1] + data_sum_x2[y, x]
                memory_sum_y2[y, x] = memory_sum_y2[y, x - 1] + data_sum_y2[y, x]
                    

        # first column
        for x in range(1):
            for y in range(1, height):
                memory_sum_x[y, x]  = memory_sum_x[y - 1, x] + data_sum_x[y, x]
                memory_sum_y[y, x]  = memory_sum_y[y - 1, x] + data_sum_y[y, x]
                memory_sum_xy[y, x] = memory_sum_xy[y - 1, x] + data_sum_xy[y, x]
                memory_sum_x2[y, x] = memory_sum_x2[y - 1, x] + data_sum_x2[y, x]
                memory_sum_y2[y, x] = memory_sum_y2[y - 1, x] + data_sum_y2[y, x]
                
        for y in range(1, height): 
            for x in range(1, width):
                memory_sum_x[y, x]  = memory_sum_x[y, x - 1] + memory_sum_x[y - 1, x] - \
                                      memory_sum_x[y - 1, x - 1] + data_sum_x[y, x] 
                memory_sum_y[y, x]  = memory_sum_y[y, x - 1] + memory_sum_y[y - 1, x] - \
                                      memory_sum_y[y - 1, x - 1] + data_sum_y[y, x]
                memory_sum_xy[y, x] = memory_sum_xy[y, x - 1] + memory_sum_xy[y - 1, x] - \
                                      memory_sum_xy[y - 1, x - 1] + data_sum_xy[y, x]
                memory_sum_x2[y, x] = memory_sum_x2[y, x - 1] + memory_sum_x2[y - 1, x] - \
                                      memory_sum_x2[y - 1, x - 1] + data_sum_x2[y, x]
                memory_sum_y2[y, x] = memory_sum_y2[y, x - 1] + memory_sum_y2[y - 1, x] - \
                                      memory_sum_y2[y - 1, x - 1] + data_sum_y2[y, x]
    
    
        for y in range(kernel_half, height-kernel_half):        
            for x in range(kernel_half, width-kernel_half):
            
                x0 = x - kernel_half
                x1 = x + kernel_half                
                y0 = y - kernel_half
                y1 = y + kernel_half
                
                sum_x = memory_sum_x[y1, x1] - memory_sum_x[y1, x0 - 1] - \
                        memory_sum_x[y0 - 1, x1] + memory_sum_x[y0 - 1, x0 - 1]
                
                sum_y = memory_sum_y[y1, x1] - memory_sum_y[y1, x0 - 1] - \
                        memory_sum_y[y0 - 1, x1] + memory_sum_y[y0 - 1, x0 - 1]
                
                sum_xy = memory_sum_xy[y1, x1] - memory_sum_xy[y1, x0 - 1] - \
                         memory_sum_xy[y0 - 1, x1] + memory_sum_xy[y0 - 1, x0 - 1]
                
                sum_x2 = memory_sum_x2[y1, x1] - memory_sum_x2[y1, x0 - 1] - \
                         memory_sum_x2[y0 - 1, x1] + memory_sum_x2[y0 - 1, x0 - 1]
                
                sum_y2 = memory_sum_y2[y1, x1] - memory_sum_y2[y1, x0 - 1] - \
                         memory_sum_y2[y0 - 1, x1] + memory_sum_y2[y0 - 1, x0 - 1]
                
                              
                corr = (n*sum_xy - sum_x*sum_y) / \
                       (np.sqrt(n*sum_x2 - sum_x**2) * np.sqrt(n*sum_y2 - sum_y**2))
                corr_space[j, y, x] = 1.0 - corr
                
                
    # tìm j tại đó cost có giá trị min
    for y in range(kernel_half, height-kernel_half):        
        for x in range(kernel_half, width-kernel_half):
            disparity = 0
            cost_min  = 1.0 # a large number            
            for j in range(disparity_range):
                cost = corr_space[j, y, x]                                
                if cost < cost_min:
                    cost_min = cost
                    disparity = j                  
            
            # gán j cho cost_min vào disaprity map
            depth[y, x] = int(disparity * scale)
                                
    # chuyển dữ liệu từ ndarray sang kiểu Image và lưu xuống file
    Image.fromarray(depth).save('Aloe/Disparity_map_corr.png')
    
disparity_range = 64 # cho cặp ảnh Tsukuba
kernel_size = 9
stereo_matching_corr("Aloe/Aloe_left_1.png", "Aloe/Aloe_right_2.png", kernel_size, disparity_range)

. 

C:\installed\lib\site-packages\ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in sqrt


. . . . . . . . . . 

C:\installed\lib\site-packages\ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in double_scalars


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 